In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


# Path to the dataset directory
dataset_dir = 'Dataset'

# Custom Dataset class
class FaceDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_dict = {}
        self._prepare_data()

    def _prepare_data(self):
        subfolders = [f for f in os.listdir(self.dataset_dir) if os.path.isdir(os.path.join(self.dataset_dir, f))]
        for idx, subfolder in enumerate(subfolders):
            self.label_dict[idx] = subfolder
            image_files = os.listdir(os.path.join(self.dataset_dir, subfolder))
            for image_file in image_files:
                self.data.append(os.path.join(self.dataset_dir, subfolder, image_file))
                self.labels.append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Triplet Loss function
def triplet_loss(anchor, positive, negative, margin=1.0):
    pos_dist = (anchor - positive).pow(2).sum(1)
    neg_dist = (anchor - negative).pow(2).sum(1)
    loss = F.relu(pos_dist - neg_dist + margin)
    return loss.mean()

# FaceNet model
class FaceNet(nn.Module):
    def __init__(self, embedding_dim=128):
        super(FaceNet, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, embedding_dim)

    def forward(self, x):
        return self.base_model(x)

# Prepare triplets
def create_triplets(dataset, indices, num_triplets):
    triplets = []
    labels = [dataset.labels[i] for i in indices]

    for _ in range(num_triplets):
        anchor_label = random.choice(labels)
        positive_label = anchor_label
        negative_label = random.choice([label for label in labels if label != anchor_label])

        anchor_indices = [i for i, label in enumerate(labels) if label == anchor_label]
        positive_indices = [i for i, label in enumerate(labels) if label == positive_label]
        negative_indices = [i for i, label in enumerate(labels) if label == negative_label]

        anchor_idx = random.choice(anchor_indices)
        positive_idx = random.choice(positive_indices)
        negative_idx = random.choice(negative_indices)

        triplets.append((indices[anchor_idx], indices[positive_idx], indices[negative_idx]))

    return triplets

# Training function
def train_model(model, dataset, train_indices, criterion, optimizer, num_epochs=25, batch_size=32):
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        triplets = create_triplets(dataset, train_indices, len(train_indices) // batch_size)

        for anchor_idx, positive_idx, negative_idx in triplets:
            anchor_img, anchor_label = dataset[anchor_idx]
            positive_img, positive_label = dataset[positive_idx]
            negative_img, negative_label = dataset[negative_idx]

            anchor_img = anchor_img.unsqueeze(0).to(device)
            positive_img = positive_img.unsqueeze(0).to(device)
            negative_img = negative_img.unsqueeze(0).to(device)

            optimizer.zero_grad()
            anchor_out = model(anchor_img)
            positive_out = model(positive_img)
            negative_out = model(negative_img)

            loss = criterion(anchor_out, positive_out, negative_out)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(triplets)}')

# Evaluation function
def evaluate_model(model, dataset, test_indices):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx in test_indices:
            img, label = dataset[idx]
            img = img.unsqueeze(0).to(device)
            output = model(img)
            _, predicted = torch.max(output, 1)
            total += 1
            if predicted == label:
                correct += 1
    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load data
dataset = FaceDataset(dataset_dir, transform=transform)

# Split dataset into training and testing sets
dataset_size = len(dataset)
indices = list(range(dataset_size))
random.shuffle(indices)
split = int(0.2 * dataset_size)
train_indices, test_indices = indices[split:], indices[:split]

# Initialize model, criterion and optimizer
model = FaceNet(embedding_dim=128).to(device)
criterion = triplet_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, dataset, train_indices, criterion, optimizer, num_epochs=10, batch_size=32)

# Evaluate the model
evaluate_model(model, dataset, test_indices)

# Save the model
torch.save(model.state_dict(), 'facenet_model.pth')

print("Training complete. Model saved as facenet_model.pth")


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [1/10], Loss: 0.9672231755473397
Epoch [2/10], Loss: 1.05179509926926
Epoch [3/10], Loss: 0.9446113678542051
Epoch [4/10], Loss: 1.062917786565694
Epoch [5/10], Loss: 1.1100957149809056
Epoch [6/10], Loss: 1.4247461638667367
Epoch [7/10], Loss: 1.1072576506571337
Epoch [8/10], Loss: 1.0114685175093738
Epoch [9/10], Loss: 1.161357037045739
Epoch [10/10], Loss: 1.0294747054576874
Accuracy on test set: 0.00%
Training complete. Model saved as facenet_model.pth


In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')



# Path to the dataset directory
dataset_dir = 'Dataset'

# Custom Dataset class
class FaceDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_dict = {}
        self._prepare_data()

    def _prepare_data(self):
        subfolders = [f for f in os.listdir(self.dataset_dir) if os.path.isdir(os.path.join(self.dataset_dir, f))]
        for idx, subfolder in enumerate(subfolders):
            self.label_dict[idx] = subfolder
            image_files = os.listdir(os.path.join(self.dataset_dir, subfolder))
            for image_file in image_files:
                self.data.append(os.path.join(self.dataset_dir, subfolder, image_file))
                self.labels.append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Triplet Loss function
def triplet_loss(anchor, positive, negative, margin=1.0):
    pos_dist = (anchor - positive).pow(2).sum(1)
    neg_dist = (anchor - negative).pow(2).sum(1)
    loss = F.relu(pos_dist - neg_dist + margin)
    return loss.mean()

# Siamese Network
class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim=128):
        super(SiameseNetwork, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, embedding_dim)

    def forward(self, x):
        return self.base_model(x)

# Prepare triplets
def create_triplets(dataset, indices, num_triplets):
    triplets = []
    labels = [dataset.labels[i] for i in indices]

    for _ in range(num_triplets):
        anchor_label = random.choice(labels)
        positive_label = anchor_label
        negative_label = random.choice([label for label in labels if label != anchor_label])

        anchor_indices = [i for i, label in enumerate(labels) if label == anchor_label]
        positive_indices = [i for i, label in enumerate(labels) if label == positive_label]
        negative_indices = [i for i, label in enumerate(labels) if label == negative_label]

        anchor_idx = random.choice(anchor_indices)
        positive_idx = random.choice(positive_indices)
        negative_idx = random.choice(negative_indices)

        triplets.append((indices[anchor_idx], indices[positive_idx], indices[negative_idx]))

    return triplets

# Training function
def train_model(model, dataset, train_indices, criterion, optimizer, num_epochs=15, batch_size=32):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        triplets = create_triplets(dataset, train_indices, len(train_indices) // batch_size)

        for anchor_idx, positive_idx, negative_idx in triplets:
            anchor_img, anchor_label = dataset[anchor_idx]
            positive_img, positive_label = dataset[positive_idx]
            negative_img, negative_label = dataset[negative_idx]

            anchor_img = anchor_img.unsqueeze(0).to(device)
            positive_img = positive_img.unsqueeze(0).to(device)
            negative_img = negative_img.unsqueeze(0).to(device)

            optimizer.zero_grad()
            anchor_out = model(anchor_img)
            positive_out = model(positive_img)
            negative_out = model(negative_img)

            loss = criterion(anchor_out, positive_out, negative_out)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(triplets)}')

# Evaluation function
def evaluate_model(model, dataset, test_indices):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for idx in test_indices:
            img, label = dataset[idx]
            img = img.unsqueeze(0).to(device)
            output = model(img)
            _, predicted = torch.max(output, 1)
            total += 1
            if predicted == label:
                correct += 1
    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load data
dataset = FaceDataset(dataset_dir, transform=transform)

# Split dataset into training and testing sets
dataset_size = len(dataset)
indices = list(range(dataset_size))
random.shuffle(indices)
split = int(0.2 * dataset_size)
train_indices, test_indices = indices[split:], indices[:split]

# Initialize model, criterion and optimizer
model = SiameseNetwork(embedding_dim=128).to(device)
criterion = triplet_loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, dataset, train_indices, criterion, optimizer, num_epochs=15, batch_size=32)

# Evaluate the model
evaluate_model(model, dataset, test_indices)

# Save the model
torch.save(model.state_dict(), 'siamese_model.pth')

print("Training complete. Model saved as siamese_model.pth")


Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [1/15], Loss: 1.015222733671015
Epoch [2/15], Loss: 0.9012248123233969
Epoch [3/15], Loss: 1.2038735638965259
Epoch [4/15], Loss: 1.0264976295557888
Epoch [5/15], Loss: 1.0267436341805891
Epoch [6/15], Loss: 1.0217152563008396
Epoch [7/15], Loss: 1.0064509768377652
Epoch [8/15], Loss: 1.0649821121584286
Epoch [9/15], Loss: 1.035891981287436
Epoch [10/15], Loss: 1.0111893117427826
Epoch [11/15], Loss: 1.173291409557516
Epoch [12/15], Loss: 1.3129462410103192
Epoch [13/15], Loss: 1.0827827114950528
Epoch [14/15], Loss: 1.0325284518978812
Epoch [15/15], Loss: 1.013532045212659
Accuracy on test set: 0.00%
Training complete. Model saved as siamese_model.pth


In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image
from sklearn.metrics import accuracy_score

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Path to the dataset directory
dataset_dir = 'Dataset'

# Custom Dataset class
class FaceDataset(Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_dict = {}
        self._prepare_data()

    def _prepare_data(self):
        subfolders = [f for f in os.listdir(self.dataset_dir) if os.path.isdir(os.path.join(self.dataset_dir, f))]
        for idx, subfolder in enumerate(subfolders):
            self.label_dict[idx] = subfolder
            image_files = os.listdir(os.path.join(self.dataset_dir, subfolder))
            for image_file in image_files:
                self.data.append(os.path.join(self.dataset_dir, subfolder, image_file))
                self.labels.append(idx)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Classification Model
class ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Training function
def train_model(model, dataloader, criterion, optimizer, num_epochs=15):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader)}')

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
    accuracy = accuracy_score(all_labels, all_predictions)
    print(f'Accuracy on test set: {accuracy * 100:.2f}%')

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
])

# Load data
dataset = FaceDataset(dataset_dir, transform=transform)

# Split dataset into training and testing sets
dataset_size = len(dataset)
indices = list(range(dataset_size))
random.shuffle(indices)
split = int(0.2 * dataset_size)
train_indices, test_indices = indices[split:], indices[:split]

train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
test_sampler = torch.utils.data.SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler)

# Initialize model, criterion and optimizer
num_classes = len(set(dataset.labels))
model = ClassificationModel(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_model(model, test_loader)

# Save the model
torch.save(model.state_dict(), 'classification_model.pth')

print("Training complete. Model saved as classification_model.pth")


Using device: cuda
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 95.6MB/s]


Epoch [1/10], Loss: 1.9307219033891505
Epoch [2/10], Loss: 1.4222906204787167
Epoch [3/10], Loss: 1.1320409070361743
Epoch [4/10], Loss: 0.9518934732133691
Epoch [5/10], Loss: 0.7382479845122858
Epoch [6/10], Loss: 0.6696929017251189
Epoch [7/10], Loss: 0.4950988512824882
Epoch [8/10], Loss: 0.4343115006658164
Epoch [9/10], Loss: 0.40990511361848225
Epoch [10/10], Loss: 0.31777152029628103
Accuracy on test set: 73.79%
Training complete. Model saved as classification_model.pth


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

# Initialize Focal Loss
criterion_focal = FocalLoss()

# Train the model with Focal Loss
train_model(model, train_loader, criterion_focal, optimizer, num_epochs=15)

# Evaluate the model
evaluate_model(model, test_loader)


Epoch [1/15], Loss: 0.030061515217477627
Epoch [2/15], Loss: 0.008487695157369175
Epoch [3/15], Loss: 0.0045405383693419435
Epoch [4/15], Loss: 0.004172254944850548
Epoch [5/15], Loss: 0.003057711627496412
Epoch [6/15], Loss: 0.0027775224863158987
Epoch [7/15], Loss: 0.0024320023746068646
Epoch [8/15], Loss: 0.0034137442170586755
Epoch [9/15], Loss: 0.0017686971932458948
Epoch [10/15], Loss: 0.001936071208505878
Epoch [11/15], Loss: 0.0021383379813125266
Epoch [12/15], Loss: 0.002047248408209932
Epoch [13/15], Loss: 0.0015627399113162194
Epoch [14/15], Loss: 0.0012899494082680783
Epoch [15/15], Loss: 0.0013093628326301273
Accuracy on test set: 83.19%


In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.criterion = nn.KLDivLoss(reduction='batchmean')

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        true_dist = pred.data.clone()
        true_dist.fill_(self.smoothing / (self.cls - 1))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return self.criterion(pred, true_dist)

# Initialize Label Smoothing Loss
criterion_label_smoothing = LabelSmoothingLoss(num_classes, smoothing=0.1)

# Train the model with Label Smoothing Loss
train_model(model, train_loader, criterion_label_smoothing, optimizer, num_epochs=15)

# Evaluate the model
evaluate_model(model, test_loader)


Epoch [1/15], Loss: 0.255461624738845
Epoch [2/15], Loss: 0.14716044000603937
Epoch [3/15], Loss: 0.1359572805125605
Epoch [4/15], Loss: 0.11123548617417162
Epoch [5/15], Loss: 0.10289283968846906
Epoch [6/15], Loss: 0.0860254265029322
Epoch [7/15], Loss: 0.06575526686554606
Epoch [8/15], Loss: 0.05750623061745004
Epoch [9/15], Loss: 0.04504436779428612
Epoch [10/15], Loss: 0.04200073780322617
Epoch [11/15], Loss: 0.035760038447650994
Epoch [12/15], Loss: 0.04096733308820562
Epoch [13/15], Loss: 0.036847452336752955
Epoch [14/15], Loss: 0.05241543583741242
Epoch [15/15], Loss: 0.0771996345879002
Accuracy on test set: 82.34%


In [ ]:
# Using Cross Entropy Loss
criterion = nn.CrossEntropyLoss()

# Using Focal Loss
# criterion = FocalLoss()

# Using Label Smoothing Loss
# criterion = LabelSmoothingLoss(num_classes, smoothing=0.1)

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=15)

# Evaluate the model
evaluate_model(model, test_loader)


Epoch [1/15], Loss: 0.06814385099675167
Epoch [2/15], Loss: 0.6617035906423222
Epoch [3/15], Loss: 0.7528125867247581
Epoch [4/15], Loss: 0.5073415616696532
Epoch [5/15], Loss: 0.33842935430055315
Epoch [6/15], Loss: 0.23729107782922007
Epoch [7/15], Loss: 0.14329142788086424
Epoch [8/15], Loss: 0.11036824316463688
Epoch [9/15], Loss: 0.08659483543173833
Epoch [10/15], Loss: 0.139278515284373
Epoch [11/15], Loss: 0.1230071563358334
Epoch [12/15], Loss: 0.13822178161618384
Epoch [13/15], Loss: 0.11222686830230734
Epoch [14/15], Loss: 0.12000984958881004
Epoch [15/15], Loss: 0.11160592554899101
Accuracy on test set: 78.06%
